In [ ]:
import torch
from torchvision.models import resnet18
from torchvision import transforms
from PIL import Image
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.transform import from_origin


In [ ]:

# Definir a arquitetura do modelo
model = resnet18(num_classes=2)  # Duas classes: Restinga (0) e Areia (1)
# Carregar os pesos do modelo
model.load_state_dict(torch.load('resultado/resnet18_modelo_classificado_restinga_areia.pth'))
# Colocar o modelo em modo de avaliação
model.eval()

# Definir as transformações aplicadas durante o treinamento
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:

# Função para prever a classe de uma imagem, com um valor adicional para "nenhuma classe"
def predict(image, threshold=0.95):  # Aumentado o threshold para maior confiabilidade
    image = transform(image).unsqueeze(0)  # Adicionar dimensão do batch
    with torch.no_grad():
        output = model(image)
        probabilities = torch.softmax(output, dim=1)  # Obter probabilidades
        max_prob, predicted = torch.max(probabilities, 1)

        # Verificar se a probabilidade é menor que o limiar
        if max_prob.item() < threshold:
            return 2  # Valor para "nenhuma classe detectada"
        return predicted.item()


In [ ]:
# Processar o GeoTIFF
with rasterio.open('dados/DJI_0128.tif') as src:
    res = src.res[0]  # Resolução espacial (tamanho do pixel em metros)
    window_size = int(1 / res)  # Tamanho da janela em pixels para 1x1 metro
    width = src.width
    height = src.height
    mask = np.zeros((height // window_size, width // window_size), dtype=np.uint8)

    # Iterar sobre a imagem em passos de 1x1 metro
    for i in range(0, height, window_size):
        for j in range(0, width, window_size):
            if i + window_size > height or j + window_size > width:
                continue

            # Ler a janela correspondente
            window = Window(j, i, window_size, window_size)
            segment = src.read(window=window)

            # Verificar se há um canal alpha (transparência)
            if segment.shape[0] == 4:  # Se houver 4 bandas (R, G, B, A)
                alpha_channel = segment[3, :, :]  # Extrair o canal alpha
                if np.all(alpha_channel == 0):  # Se for totalmente transparente
                    mask[i // window_size, j // window_size] = 2
                    continue  # Pular para o próximo pixel

            # Garantir que segment tenha apenas 3 bandas (R, G, B)
            if segment.shape[0] >= 3:
                segment = segment[:3]  # Selecionar as 3 primeiras bandas (R, G, B)
            else:
                raise ValueError("O segmento não possui bandas suficientes para RGB.")

            # Converter para imagem PIL
            segment_image = Image.fromarray(np.moveaxis(segment, 0, -1))
            # Prever a classe
            class_id = predict(segment_image, threshold=0.95)
            # Atualizar a máscara
            mask[i // window_size, j // window_size] = class_id

# Definir a transformação geográfica para a máscara
transform = from_origin(src.bounds.left, src.bounds.top, window_size * res, window_size * res)

In [ ]:
# Salvar a máscara como GeoTIFF
with rasterio.open(
    'resultado/DJI_0128_mascara_restinga_areia.tif',
    'w',
    driver='GTiff',
    height=mask.shape[0],
    width=mask.shape[1],
    count=1,
    dtype=mask.dtype,
    crs=src.crs,
    transform=transform,
) as dst:
    dst.write(mask, 1)